In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres']=movies['genres'].apply(convert)

In [8]:
movies['keywords']=movies['keywords'].apply(convert)

In [9]:
def convert_cast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if(counter!=3):
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [10]:
movies['cast']=movies['cast'].apply(convert_cast)

In [11]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            L.append(i['name'])
            break
    return L

In [12]:
movies['crew']=movies['crew'].apply(fetch_director)

In [13]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [14]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [15]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [16]:
new_df=movies[['movie_id','title','tags']]

In [17]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_7060\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [18]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_7060\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [19]:
import nltk

In [20]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [21]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [22]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_7060\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [24]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity=cosine_similarity(vectors)

In [27]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    movies_list=sorted(list(enumerate(similarity[movie_index])),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [28]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [29]:
import pickle

In [32]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [36]:
pickle.dump(similarity,open('similarity.pkl','wb'))